# 제너레이터 ── 메모리 효율이 높은 이터러블한 객체

## 제너레이터의 구체적인 예시

In [ ]:


# yield를 포함한 함수는 제너레이터가 됨
def inf(n):
    while True:
        yield n

typ        

## 제너레이터 구현

### 제너레이터 함수 ── 함수처럼 작성함

In [ ]:
def gen_function(n):
    print('start')
    while n:
        print(f'yield: {n}')
        yield n  # 여기에서 일시 중단됨
        n -= 1

In [3]:
# 반환값은 제너레이터 이터레이터
gen = gen_function(2)
gen

<generator object gen_function at 0x7fa5b8a566d0>

In [4]:
# 내장 함수 next()에 전달하면 
# __next__()가 호출됨
next(gen)

start
yield: 2


2

In [5]:
next(gen)

yield: 1


1

In [6]:
next(gen)

StopIteration: 

In [7]:
def gen_function(n):
    while n:
        yield n
        n -= 1

In [8]:
# for 문에서의 이용
for i in gen_function(2):
    print(i)

2
1


In [9]:
# 리스트 컴프리헨션에서의 이용
[i for i in gen_function(5)]

[5, 4, 3, 2, 1]

In [10]:
# 이터러블을 받는 함수에 전달함
max(gen_function(5))

5

### 제너레이터 식 ── 리스트 컴프리헨션을 이용해 작성

In [11]:
x = [1, 2, 3, 4, 5]

In [12]:
# 리스트 컴프리헨션
listcomp = [i**2 for i in x]
listcomp  # 모든 엘리먼트가 메모리 상에 전개됨

[1, 4, 9, 16, 25]

In [13]:
# 제너레이터 식 
gen = (i**2 for i in x)
gen  # 각 엘리먼트는 필요할 때까지 계산되지 않음

<generator object <genexpr> at 0x7fa5b8a7f430>

In [14]:
# 리스트로 만들면 가장 마지막 엘리먼트까지 계산됨
list(gen)

[1, 4, 9, 16, 25]

In [15]:
x = [1, 2, 3, 4, 5]

# max((i**3 for i in x))과 같음
max(i**3 for i in x)

125

### yield from 식 ── 서브 제너레이터로 처리를 이첩

In [16]:
def chain(iterables):
    for iterable in iterables:
        for v in iterable:
            yield v

In [17]:
iterables = ('python', 'book')

In [18]:
list(chain(iterables))

['p', 'y', 't', 'h', 'o', 'n', 'b', 'o', 'o', 'k']

In [19]:
def chain(iterables):
    for iterable in iterables:
        yield from (v for v in iterable)

In [20]:
list(chain(iterables))

['p', 'y', 't', 'h', 'o', 'n', 'b', 'o', 'o', 'k']

## 제너레이터 이용 시 주의점

In [21]:
def gen(n):
    while n:
        yield n
        n -= 1

In [22]:
# zip()에 리스트와 제너레이터를 동시에 전달함
x = [1, 2, 3, 4, 5]
[i for i in zip(x, gen(5))]

[(1, 5), (2, 4), (3, 3), (4, 2), (5, 1)]

In [23]:
# filter()에 제너레이터를 전달함
odd = filter(lambda v: v % 2 == 1, gen(5))
[i for i in odd]

[5, 3, 1]

### len()으로 이용할 때

In [24]:
len(gen(5))

TypeError: object of type 'generator' has no len()

In [25]:
len(list(gen(5)))

5

In [26]:
# 값을 무한히 반환하는 제너레이터
g = gen(-1)

### 여러 차례 이용할 때

In [27]:
g = gen(4)
len(list(g))

4

In [28]:
len(list(g))

0

In [29]:
g = gen(4)
list_nums = list(gen(4))
len(list_nums)

4

In [30]:
len(list_nums)

4

## 제너레이터 실제 사례 ── 파일 내용 변환하기

In [31]:
def convert(line):
    return line.upper()

In [32]:
def reader(src):
    with open(src) as f:
        for line in f:
            yield line

In [33]:
def writer(dest, reader):
    with open(dest, 'w') as f:
        for line in reader:
            f.write(convert(line))

## 기타 유스 케이스

# 데커레이터 ── 함수나 클래스 이름에 처리 추가

## 데커레이터의 구체적인 사례

### functools.lru\_cache() ── 함수 결과를 캐쉬하는 함수 데커레이터

In [34]:
from functools import lru_cache
from time import sleep

# 최근 호출한 인수와 결과를 최대 32회까지 캐쉬
@lru_cache(maxsize=32)
def heavy_funcion(n):
    sleep(3)  # 무거운 처리 시뮬레이션
    return n + 1

In [35]:
# 최초에는 시간이 걸림
heavy_funcion(2)

3

In [36]:
# 캐쉬된 상태에서는 즉시 결과를 얻을 수 있음
heavy_funcion(2)

3

### dataclasses.dataclass() ── 자주 하는 처리를 자동으로 추가하는 클래스 데커레이터

In [37]:
from dataclasses import dataclass
@dataclass(frozen=True)
class Fruit:
    name: str  # 타입 힌트를 붙여 속성을 정의
    price: int = 0  # 초기값도 지정

In [38]:
# __init__()나 __repr__()가 추가되어 있음
apple = Fruit(name='apple', price=128)
apple

Fruit(name='apple', price=128)

In [39]:
# frozen=True이므로 읽기 전용임
apple.price = 256

FrozenInstanceError: cannot assign to field 'price'

## 데커레이터 구현

### 간단한 데커레이터

In [40]:
# 데커레이트할 함수를 받음
def deco1(f):
    print('deco1 called')
    def wrapper():
        print('before exec')
        v = f()  # 원래 함수를 실행
        print('after exec')
        return v
    return wrapper

In [41]:
# 데커레이터 함수 정의 시 실행됨
@deco1
def func():
    print('exec')
    return 1

deco1 called


In [42]:
# 이미 deco1(func)의 결과로 대체되어 있음
func.__name__

'wrapper'

In [43]:
func()

before exec
exec
after exec


1

In [44]:
@deco1
def func(x, y):
    print('exec')
    return x, y

deco1 called


In [45]:
# deco1 안의 v = f() 행에서 에러가 발생함
func(1, 2)

TypeError: wrapper() takes 0 positional arguments but 2 were given

### 인수를 받는 함수 데커레이터

In [46]:
def deco2(f):
    # 새로운 함수가 인수를 받음
    def wrapper(*args, **kwargs):
        print('before exec')
        # 인수를 전달해 원래 함수를 실행
        v = f(*args, **kwargs)
        print('after exec')
        return v
    return wrapper

In [47]:
@deco2
def func(x, y):
    print('exec')
    return x, y

In [48]:
func(1, 2)

before exec
exec
after exec


(1, 2)

### 데커레이터 자체가 인수를 받는 데커레이터

In [49]:
# 인수 z를 받음
def deco3(z):
    # deco2()와 같음
    def _deco3(f):
        def wrapper(*args, **kwargs):
            # 여기에서 z를 참조할 수 있음
            print('before exec', z)
            v = f(*args, **kwargs)
            print('after exec', z)
            return v
        return wrapper
    return _deco3  # 데커레이터를 반환

In [50]:
# deco3(z=3)의 반환값이 데커레이터의 실체, 
# 즉 func = deco3(z=3)(func)와 같음 
@deco3(z=3)
def func(x, y):
    print('exec')
    return x, y

In [51]:
# z에 전달한 값은 유지되고 있음
func(1, 2)

before exec 3
exec
after exec 3


(1, 2)

### 여러 데커레이터를 동시에 이용

In [52]:
# 여러 데커레이터를 이용
@deco3(z=3)
@deco3(z=4)
def func(x, y):
    print('exec')
    return x, y

In [53]:
# @deco3(z=4)가 적용된 결과에
# @deco3(z=3)이 적용됨
func(1, 2)

before exec 3
before exec 4
exec
after exec 4
after exec 3


(1, 2)

### functools.wraps()로 데커레이터 결함 해결

In [54]:
from functools import wraps
def deco4(f):
    @wraps(f)  # 원래 함수를 인수로 받는 데커레이터
    def wrapper(*args, **kwargs):
        print('before exec')
        v = f(*args, **kwargs)
        print('after exec')
        return v
    return wrapper

In [55]:
@deco4
def func():
    """func입니다"""
    print('exec')

In [56]:
func.__name__

'func'

In [57]:
func.__doc__

'func입니다'

## 데커레이터 실제 사례 ── 처리 시간 측정

In [58]:
from functools import wraps
import time
def elapsed_time(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        start = time.time()
        v = f(*args, **kwargs)
        print(f"{f.__name__}: {time.time() - start}")
        return v
    return wrapper

In [59]:
# 0부터 n-1까지의 총합을 계산하는 함수
@elapsed_time
def func(n):
    return sum(i for i in range(n))

In [60]:
# func() 실행 결과 표시
# f-string에서 콤마(,）로 구분해 여러값을 지정
print(f'{func(1000000)=:,}')

func: 0.07166790962219238
func(1000000)=499,999,500,000


In [61]:
print(f'{func(10000000)=:,}')

func: 0.5436811447143555
func(10000000)=49,999,995,000,000


# 컨텍스트 관리자 ── with 문 앞뒤에서 처리를 실행하는 객체

## 컨텍스트 관리자의 구체적인 예시

In [62]:
# 두 번쩨 인수로 쓰기 모드 지정
with open('some.txt', 'w') as f:
    f.write('python')

In [63]:
f.closed

True

In [64]:
with open('some.txt', 'w') as f:
    f.read()  # 쓰기 모드이므로 예외가 발생함

UnsupportedOperation: not readable

In [65]:
# 예외 발생 시에도 파일은 닫혀 있음
f.closed

True

In [66]:
try:
    f = open('some.txt', 'w')
    f.read()  # 쓰기 모드이므로 예외가 발생함
finally:
    f.close()

UnsupportedOperation: not readable

In [67]:
# 예외 발생 시에도 파일은 닫혀 있음
f.closed

True

## 컨텍스트 관리자 구현

### \_\_enter\_\_()、\_\_exit\_\_() ── with 문 앞뒤에서 호출되는 메서드

In [68]:
# 이 클래스의 인스턴스가 컨텍스트 관리자임
class ContextManager:
    # 전처리 구현
    def __enter__(self):
        print('__enter__ was called')
    # 후처리 구현
    def __exit__(self, exc_type, exc_value, traceback):
        print('__exit__ was called')
        print(f'{exc_type=}')
        print(f'{exc_value=}')
        print(f'{traceback=}')

In [69]:
# with 블록이 정상 종료될 때는
# __exit__()의 인수는 모두 None
with ContextManager():
    print('inside the block')

__enter__ was called
inside the block
__exit__ was called
exc_type=None
exc_value=None
traceback=None


### with 문과 예외 처리

In [70]:
# with 블록 안에서 예외가 발생한 때는
# 해당 정보가 __exit__()으로 전달됨
with ContextManager():
    1 / 0

__enter__ was called
__exit__ was called
exc_type=<class 'ZeroDivisionError'>
exc_value=ZeroDivisionError('division by zero')
traceback=<traceback object at 0x7fa5785fcdc0>


ZeroDivisionError: division by zero

### as 키워드 ── \_\_enter\_\_()의 반환값을 이용

In [71]:
class ContextManager:
    # 반환값이 as 키워드에 전달됨
    def __enter__(self):
        return 1
    def __exit__(self, exc_type, exc_value, traceback):
        pass

In [72]:
with ContextManager() as f:
    print(f)

1


In [73]:
# as 키워드 생략
with ContextManager():
    pass

In [74]:
class Point:
    def __init__(self, **kwargs):
        self.value = kwargs
    def __enter__(self):
        print('__enter__ was called')
        return self.value  # as 절로 전달됨
    def __exit__(self, exc_type, exc_value, traceback):
        print('__exit__ was called')
        print(self.value)

In [75]:
with Point(x=1, y=2) as p:
    print(p)
    p['z'] = 3

__enter__ was called
{'x': 1, 'y': 2}
__exit__ was called
{'x': 1, 'y': 2, 'z': 3}


### contextlib.contextmanager로 간단히 구현

In [76]:
from contextlib import contextmanager
@contextmanager
def point(**kwargs):
    print('__enter__ was called')
    value = kwargs
    try:
        # 이곳부터 전처리
        # value를 as 키워드에 전달
        yield value
        # 이곳부터 아래가 후처리
    except Exception as e:
        # 에러 시는 이곳도 호출됨
        print(e)
        raise
    finally:
        print('__exit__ was called')
        print(value)

In [77]:
with point(x=1, y=2) as p:
    print(p)
    p['z'] = 3

__enter__ was called
{'x': 1, 'y': 2}
__exit__ was called
{'x': 1, 'y': 2, 'z': 3}


## 컨텍스트 관리자 실제 사례 ── 일시적인 로깅 레벨 변경

In [78]:
!cat debug_context.py

import logging
from contextlib import contextmanager

logger = logging.getLogger(__name__)
logger.addHandler(logging.StreamHandler())

# 기본값이 INFO 레벨이므로, DEBUG 레벨 로그는 무시됨
logger.setLevel(logging.INFO)

@contextmanager
def debug_context():
    level = logger.level
    try:
        # 로깅 레벨을 변경함
        logger.setLevel(logging.DEBUG)
        yield
    finally:
        # 원래 로깅 레벨로 되돌림
        logger.setLevel(level)

def main():
    logger.info('before: info log')
    logger.debug('before: debug log')

    # DEBUG 로그를 볼 때의 처리를 with 문 블록 안에서 실행함
    with debug_context():
        logger.info('inside the block: info log')
        logger.debug('inside the block: debug log')

    logger.info('after: info log')
    logger.debug('after: debug log')

if __name__ == '__main__':
    main()

In [79]:
!python3 debug_context.py

before: info log
inside the block: info log
inside the block: debug log
after: info log


## 기타 유스 케이스

# 디스크립터 ── 속성 처리를 클래스로 이첩

## 디스크립터의 구체적인 예시

In [80]:
# 디스크립터가 가진 메서드가 정의되어 있음
dir(property())

['__class__',
 '__delattr__',
 '__delete__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__isabstractmethod__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__set__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'deleter',
 'fdel',
 'fget',
 'fset',
 'getter',
 'setter']

In [81]:
# property는 실체는 클래스로 정의되어 있음
type(property())

property

In [82]:
class A:
    def f(self):
        pass

In [83]:
# 디스크립터가 가진 메서드가 정의되어 있음
dir(A.f)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [84]:
type(A.f)

function

## 디스크립터 구현

### \_\_set\_\_() 구현 ── 데이터 디스크립터

In [85]:
# __set__()을 가진 클래스는 데이터 디스크립터임
class TextField:
    def __set_name__(self, owner, name):
        print(f'__set_name__ was called')
        print(f'{owner=}, {name=}')
        self.name = name
    def __set__(self, instance, value):
        print('__set__ was called')
        if not isinstance(value, str):
            raise AttributeError('must be str')
        # .표기법이 아닌 속성 딕셔너리를 사용해 저장
        instance.__dict__[self.name] = value
    def __get__(self, instance, owner):
        print('__get__ was called')
        return instance.__dict__[self.name]

In [86]:
class Book:
    title = TextField()

__set_name__ was called
owner=<class '__main__.Book'>, name='title'


In [87]:
book = Book()

In [88]:
# 대입 시에는 __set__()이 호출됨
book.title = 'Python Practice Book'

__set__ was called


In [89]:
# 취득 시에는 __get__()이 호출됨
book.title

__get__ was called


'Python Practice Book'

In [90]:
# 다른 인스턴스를 작성해서 대입
notebook = Book()

In [91]:
notebook.title = 'Notebook'

__set__ was called


In [92]:
# 각 데이터를 유지하고 있음
book.title

__get__ was called


'Python Practice Book'

In [93]:
notebook.title

__get__ was called


'Notebook'

In [94]:
# 문자열 이외는 대입할 수 없음
book.title = 123

__set__ was called


AttributeError: must be str

### \_\_get\_\_()만 구현 ── 비데이터 디스크립터

In [95]:
# __get__()만 있다면 비데이터 디스크립터
class TextField:
    def __init__(self, value):
        if not isinstance(value, str):
            raise AttributeError('must be str')
        self.value = value
    def __set_name__(self, owner, name):
        print(f'__set_name__ was called')
        print(f'{owner=}, {name=}')
        self.name = name
    def __get__(self, instance, owner):
        print('__get__ was called')
        return self.value

In [96]:
class Book:
    title = TextField('Python Practice Book')

__set_name__ was called
owner=<class '__main__.Book'>, name='title'


In [97]:
book = Book()

In [98]:
# 대입 전 취득 시에는 __get__()가 호출됨
book.title

__get__ was called


'Python Practice Book'

In [99]:
# 대입하면 인스턴스 변수가 됨
book.title = 'Book'

In [100]:
# 인스턴스 변수가 있으면 __get__()은 호출되지 않음
book.title

'Book'

## 디스크립터 실제 사례 ── 프로퍼티 캐쉬

In [101]:
class LazyProperty:
    def __init__(self, func):
        self.func = func
        self.name = func.__name__
    def __get__(self, instance, owner):
        if not instance:
            # 클래스 변수로써 엑세스될 때의 처리
            return self
        # self.func는 함수이므로 명시적으로 인스턴스를 전달함
        v = self.func(instance)
        instance.__dict__[self.name] = v
        return v

In [102]:
TAX_RATE = 1.10

In [103]:
class Book:
    def __init__(self, raw_price):
        self.raw_price = raw_price
    @LazyProperty
    def price(self):
        print('calculate the price')
        return int(self.raw_price * TAX_RATE)

In [104]:
book = Book(1980)
book.price

calculate the price


2178

In [105]:
book.price

2178

In [106]:
Book.price

## 기타 유스 케이스

# 정리